In [31]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

In [32]:
benchmarks = pd.read_csv('benchmarks.csv')
metrics = pd.read_csv('metrics.csv')

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=benchmarks.query("type=='range'"), x='k', y='real_time', hue='tag', style='tag', alpha=0.8)
#sns.lineplot(data=benchmarks, x='k', y='real_time', hue='type', style='type', err_style=None)
plt.xlabel('k')
plt.ylabel('real time (s)')
plt.yscale('log')
plt.title('Comparison time by device and threading')
plt.legend(title='Device-Threading', bbox_to_anchor=(1.01, 1), loc='upper left')
plt.tight_layout()
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=benchmarks.query("type=='range' & k <= 20"), x='k', y='real_time', hue='tag', style='tag', alpha=0.8)
#sns.lineplot(data=benchmarks, x='k', y='real_time', hue='type', style='type', err_style=None)
plt.xlabel('k')
plt.ylabel('real time (s)')
plt.yscale('log')
plt.title('Comparison time by device and threading')
plt.legend(title='Device-Threading', bbox_to_anchor=(1.01, 1), loc='upper left')
plt.tight_layout()
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=benchmarks.query("type=='range'"), x='k', y='real_time', hue='tag', style='tag', alpha=0.8)
#sns.lineplot(data=benchmarks, x='k', y='real_time', hue='type', style='type', err_style=None)
plt.xlabel('k')
plt.ylabel('real time (s)')
plt.title('Comparison time by device and threading')
plt.legend(title='Device-Threading', bbox_to_anchor=(1.01, 1), loc='upper left')
plt.tight_layout()
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=benchmarks.query("type=='range' & k <= 20"), x='k', y='real_time', hue='tag', style='tag', alpha=0.8)
#sns.lineplot(data=benchmarks, x='k', y='real_time', hue='type', style='type', err_style=None)
plt.xlabel('k')
plt.ylabel('real time (s)')
plt.title('Comparison time by device and threading')
plt.legend(title='Device-Threading', bbox_to_anchor=(1.01, 1), loc='upper left')
plt.tight_layout()
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.lineplot(data=benchmarks.query("type=='range' & k <= 20"), x='k', y='real_time', hue='tag', style='tag', errorbar=('ci', 99), err_style='bars')
plt.xlabel('k')
plt.ylabel('real time (s)')
plt.yscale('log')
plt.title('Comparison time by device and threading (CI 99%)')
plt.legend(title='Device-Threading', bbox_to_anchor=(1.01, 1), loc='upper left')
plt.tight_layout()
plt.grid()
plt.show()

In [ ]:
# hardware speedup
cpu_single_range = metrics.query("aggregate_name=='mean' & tag=='cpu-single'")[['k', 'real_time']].reset_index(drop=True)
cpu_multi_range = metrics.query("aggregate_name=='mean' & tag=='cpu-multi'")[['k', 'real_time']].reset_index(drop=True)
gpu_single_range = metrics.query("aggregate_name=='mean' & tag=='gpu'")[['k', 'real_time']].reset_index(drop=True)
cpu_poly = metrics.query("aggregate_name=='mean' & type=='poly' & device=='cpu'")[['real_time']].reset_index(drop=True)
gpu_poly = metrics.query("aggregate_name=='mean' & type=='poly' & device=='gpu'")[['real_time']].reset_index(drop=True)

multi_speedup = pd.DataFrame({'speedup': cpu_single_range['real_time'] / cpu_multi_range['real_time'],
                              'device': 'multithreading',
                              'k': cpu_single_range['k']})
gpu_speedup = pd.DataFrame({'speedup': cpu_single_range['real_time'] / gpu_single_range['real_time'],
                              'device': 'gpu',
                              'k': cpu_single_range['k']})

gpu_poly_speedup = (cpu_poly['real_time'] / gpu_poly['real_time']).mean()
gpu_poly_speedup


In [ ]:
gpu_speedup

In [ ]:
multi_speedup

In [ ]:
gpu_to_multi_speedup = pd.DataFrame({'speedup': cpu_multi_range['real_time'] / gpu_single_range['real_time'],
                              'device': 'gpu',
                              'k': cpu_multi_range['k']})
gpu_to_multi_speedup

In [ ]:
plt.figure(figsize=(10, 6))
sns.lineplot(data=benchmarks.query("type=='range' & k <= 20"), x='k', y='real_time', hue='tag', style='tag', errorbar=('ci', 99), err_style='bars')
plt.xlabel('k')
plt.ylabel('real time (s)')
plt.title('Comparison time by device and threading (CI 99%)')
plt.legend(title='Device-Threading', bbox_to_anchor=(1.01, 1), loc='upper left')
plt.tight_layout()
plt.grid()
#plt.text(x=18, y=20, s="6.6x avg. speedup")
#plt.text(x=18, y=5, s="126.8x avg. speedup")
plt.show()

In [ ]:
# algorithm speedup
avg_poly_time_cpu = cpu_poly.mean()['real_time']
avg_poly_time_gpu = gpu_poly.mean()['real_time']
speedup = metrics.query("aggregate_name=='mean' & type=='range' & k<=20 & tag!='cpu-single'")[['k', 'real_time', 'tag']].reset_index(drop=True)
# speedup['cpu_speedup'] = avg_poly_time_cpu / speedup['real_time']
# speedup['gpu_speedup'] = avg_poly_time_gpu / speedup['real_time']
speedup.loc[speedup['tag'] == 'cpu-multi', ['speedup']] = avg_poly_time_cpu / speedup['real_time']
speedup.loc[speedup['tag'] == 'gpu', ['speedup']] = avg_poly_time_gpu / speedup['real_time']
speedup


In [ ]:
plt.figure(figsize=(10, 6))
sns.lineplot(data=speedup, x='k', y='speedup', hue='tag', style='tag', errorbar=('ci', 99), err_style='bars', markers=True)
plt.xlabel('k')
plt.ylabel('Speed-up factor')
plt.title('Range-based comparison speed-up by k and device')
plt.legend(title='Device', bbox_to_anchor=(1.01, 1), loc='upper left')
plt.tight_layout()
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.lineplot(data=benchmarks.query("type=='range'"), x='k', y='real_time', hue='tag', style='tag', errorbar=('ci', 99), err_style='bars')
plt.xlabel('k')
plt.ylabel('real time (s)')
plt.yscale('log')
plt.title('Comparison time by device and threading (CI 99%)')
plt.legend(title='Device-Threading', bbox_to_anchor=(1.01, 1), loc='upper left')
plt.tight_layout()
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.lineplot(data=benchmarks.query("type=='range'"), x='k', y='real_time', hue='tag', style='tag', errorbar=('ci', 99), err_style='bars')
plt.xlabel('k')
plt.ylabel('real time (s)')
plt.title('Comparison time by device and threading (CI 99%)')
plt.legend(title='Device-Threading', bbox_to_anchor=(1.01, 1), loc='upper left')
plt.tight_layout()
plt.grid()
#plt.text(x=22, y=16, s="6.6x avg. speedup")
#plt.text(x=80, y=10, s="126.8x avg. speedup")
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.lineplot(data=benchmarks.query('(type=="poly" | type=="range")'), x='k', y='real_time', hue='tag', style='tag', errorbar=('ci', 99), err_style='bars')
plt.xlabel('k')
plt.ylabel('real time (s)')
plt.title('Comparison time by device and type of algorithm (CI 99%)')
plt.legend(title='Device-type', bbox_to_anchor=(1.01, 1), loc='upper left')
plt.tight_layout()
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.regplot(data=benchmarks.query('type=="range" & device=="cpu" & threading=="single" & k <= 20'), x='k', y='real_time', label='cpu-single', marker='o', ci=99)
sns.regplot(data=benchmarks.query('type=="range" & device=="cpu" & threading=="multi" & k <= 20'), x='k', y='real_time', label='cpu-multi', line_kws={"ls":"-."}, marker='d', ci=99)
sns.regplot(data=benchmarks.query('type=="range" & device=="gpu" & k <= 20'), x='k', y='real_time', label='gpu', marker='x', line_kws={"ls":"--"}, ci=99)
#sns.regplot(x=pd.to_numeric(df_multi_gpu['k']), y=pd.to_numeric(df_multi_gpu['real_time']), label='gpu-multi', line_kws={"ls":":"}, marker='v', ci=99)
plt.xlabel('k')
plt.ylabel('real time (s)')
plt.title('Comparison time by device and threading (linear regression)')
plt.legend(title='Device-Threading', bbox_to_anchor=(1.01, 1), loc='upper left')
plt.tight_layout()
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.regplot(data=benchmarks.query('type=="range" & device=="cpu" & threading=="single" & k <= 20'), x='k', y='real_time', label='cpu-single', marker='o', x_estimator=np.mean, order=1, ci=99)
sns.regplot(data=benchmarks.query('type=="range" & device=="cpu" & threading=="multi" & k <= 20'), x='k', y='real_time', label='cpu-multi', line_kws={"ls":"-."}, marker='d', x_estimator=np.mean, order=1, ci=99)
sns.regplot(data=benchmarks.query('type=="range" & device=="gpu" & k <= 20'), x='k', y='real_time', label='gpu', marker='x', line_kws={"ls":"--"}, x_estimator=np.mean, order=1, ci=99)
#sns.regplot(x=pd.to_numeric(df_multi_gpu['k']), y=pd.to_numeric(df_multi_gpu['real_time']), label='gpu-multi', line_kws={"ls":":"}, marker='v', ci=99)
plt.xlabel('k')
plt.ylabel('real time (s)')
plt.title('Comparison time by device and threading (linear regression)')
plt.legend(title='Device-Threading', bbox_to_anchor=(1.01, 1), loc='upper left')
plt.tight_layout()
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.lineplot(data=benchmarks.query('type=="poly"'), x='k', y='real_time', hue='device', style='device', errorbar=('ci', 99), err_style='bars')
#sns.histplot(data=benchmarks.query('type=="poly"'), x='repetition_index', y='real_time', hue='device')
#sns.barplot(data=benchmarks.query('type=="poly"'), y='device', x='real_time', errorbar=('ci', 99))
#sns.boxplot(data=benchmarks.query('type=="poly"'), y='device', x='real_time')
plt.xlabel('k')
plt.ylabel('real time (s)')
plt.title('Comparison time by univariate polynomial (CI 99%)')
plt.legend(title='Device', bbox_to_anchor=(1.01, 1), loc='upper left')
plt.tight_layout()
plt.grid()
plt.text(x=42, y=40, s="9.1x avg. speedup")
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
#sns.lineplot(data=benchmarks.query('type=="poly"'), x='k', y='real_time', hue='device', style='device', errorbar=('ci', 99), err_style='bars')
#sns.histplot(data=benchmarks.query('type=="poly"'), x='repetition_index', y='real_time', hue='device')
ax = sns.barplot(data=benchmarks.query('type=="poly"'), y='device', x='real_time', errorbar=('ci', 99))
plt.bar_label(ax.containers[0], fmt='%.2f', padding=5)
#sns.boxplot(data=benchmarks.query('type=="poly"'), y='device', x='real_time')
plt.xlabel('real time (s)')
plt.ylabel('device')
plt.xlim(0, 300)
plt.title('Comparison time by univariate polynomial (CI 99%)')
#plt.legend(title='Device', bbox_to_anchor=(1.01, 1), loc='upper left')
plt.tight_layout()
plt.grid(axis='x')
plt.text(x=52, y=1.01, s="(9.1x avg. speedup)")
plt.show()

In [ ]:
cpu_enc = metrics.query("aggregate_name=='mean' & type=='encoding' & device=='cpu'")['real_time'].reset_index(drop=True)[0]
cpu_encr = metrics.query("aggregate_name=='mean' & type=='encryption' & device=='cpu'")['real_time'].reset_index(drop=True)[0]
cpu_dec = metrics.query("aggregate_name=='mean' & type=='decoding' & device=='cpu'")['real_time'].reset_index(drop=True)[0]
cpu_decr = metrics.query("aggregate_name=='mean' & type=='decryption' & device=='cpu'")['real_time'].reset_index(drop=True)[0]
gpu_enc = metrics.query("aggregate_name=='mean' & type=='encoding' & device=='gpu'")['real_time'].reset_index(drop=True)[0]
gpu_encr = metrics.query("aggregate_name=='mean' & type=='encryption' & device=='gpu'")['real_time'].reset_index(drop=True)[0]
gpu_dec = metrics.query("aggregate_name=='mean' & type=='decoding' & device=='gpu'")['real_time'].reset_index(drop=True)[0]
gpu_decr = metrics.query("aggregate_name=='mean' & type=='decryption' & device=='gpu'")['real_time'].reset_index(drop=True)[0]

speedup_enc = cpu_enc / gpu_enc
speedup_encr = cpu_encr / gpu_encr
speedup_dec = cpu_dec / gpu_dec
speedup_decr = cpu_decr / gpu_decr
print(speedup_enc, speedup_encr, speedup_dec, speedup_decr)

In [ ]:
data=benchmarks.query('type!="range" & type!="poly"')

plt.figure(figsize=(10, 6))
ax = sns.barplot(data=data, y='type', x=data['real_time'] * 1000, hue='device', errorbar=('ci', 99))
plt.xlabel('real time (ms)')
plt.bar_label(ax.containers[0], fmt='%.3f', padding=10)
plt.bar_label(ax.containers[1], fmt='%.3f', padding=3)
#plt.xlim(0, 0.05)
plt.ylabel('device')
plt.title('Basic operation time cost by device (CI 99%)')
plt.legend(title='Device', bbox_to_anchor=(1.01, 1), loc='upper left')
plt.tight_layout()
plt.grid(axis='x')
plt.text(x=6, y=0, s="(7.9x avg. speedup)")
plt.text(x=6, y=1, s="(2.9x avg. speedup)")
plt.text(x=32, y=2.2, s="(83.5x avg. speedup)")
plt.text(x=16, y=3.2, s="(53.3x avg. speedup)")
plt.show()

In [ ]:
print(cpu_poly['real_time'].mean() / cpu_multi_range.query("k==20.0")['real_time'])
print(gpu_poly['real_time'].mean() / gpu_single_range.query("k==20.0")['real_time'])

In [ ]:
print(cpu_poly['real_time'].mean() / cpu_multi_range.query("k==100.0")['real_time'])
print(gpu_poly['real_time'].mean() / gpu_single_range.query("k==100.0")['real_time'])

In [ ]:
plt.figure(figsize=(10, 6))
sns.lineplot(data=benchmarks.query('(type=="poly" | type=="range") & k <= 20'), x='k', y='real_time', hue='tag', style='tag', errorbar=('ci', 99), err_style='bars')
plt.xlabel('k')
plt.ylabel('real time (s)')
plt.title('Range comparison vs polynomial comparison (CI 99%)')
plt.legend(title='Device-type', bbox_to_anchor=(1.01, 1), loc='upper left')
plt.tight_layout()
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(15, 9))
sns.barplot(data=benchmarks.query('(type=="poly" | type=="range")'), x='k', y='real_time', hue='tag', errorbar=('ci', 99))
plt.xlabel('k')
plt.ylabel('real time (s)')
plt.title('Range comparison vs polynomial comparison (CI 99%)')
plt.legend(title='Device', bbox_to_anchor=(1.01, 1), loc='upper left')
plt.tight_layout()
plt.grid(axis='y')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.lineplot(data=benchmarks.query('(type=="poly" | type=="range") & k <= 20'), x='k', y='real_time', hue='tag', style='tag', errorbar=('ci', 99), err_style='bars')
plt.xlabel('k')
plt.ylabel('real time (s)')
plt.yscale('log')
plt.title('Range comparison vs polynomial comparison (CI 99%)')
plt.legend(title='Device', bbox_to_anchor=(1.01, 1), loc='upper left')
plt.tight_layout()
plt.grid()
plt.show()